# < VGG >

## - Import libaries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, utils
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np

## - Set GPU Envrionment

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## - Set Hyper-parameters
  

In [4]:
# For Training
EPOCHS     = 10
BATCH_SIZE = 256

# For Optimizer
LR         = 0.005
WD         = 0.005
MOMENTUM   = 0.9

## - Prepare the Dataset

### - DataLoader for Train / Test

In [5]:
transform_CIFAR10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root      = './dataset/CIFAR10', 
        train     = True,
        download  = True,
        transform = transform_CIFAR10),
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory = True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root      = './dataset/CIFAR10', 
        train     = False,
        download  = True,
        transform = transform_CIFAR10),
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory = True,
    num_workers=4
)

Files already downloaded and verified
Files already downloaded and verified


# **The VGG network structure**


*   The input of VGG is set to an RGB image of 224x244 size. The average RGB value is calculated for all images on the training set image, and then the image is input as an input to the VGG convolution network. A 3x3 or 1x1 filter is used, and the convolution step is fixed. . There are 3 VGG fully connected layers, which can vary from VGG11 to VGG19 according to the total number of convolutional layers + fully connected layers. The minimum VGG11 has 8 convolutional layers and 3 fully connected layers. The maximum VGG19 has 16 convolutional layers. +3 fully connected layers. In addition, the VGG network is not followed by a pooling layer behind each convolutional layer, or a total of 5 pooling layers distributed under different convolutional layers. The following figure is VGG Structure diagram:




![vgg.png](https://raw.githubusercontent.com/blurred-machine/Data-Science/master/Deep%20Learning%20SOTA/img/vgg.png)

**VGG Network Configuration**

Table 1 shows all network configurations. These networks follow the same design principles, but differ in depth.

![vggpaper](https://raw.githubusercontent.com/blurred-machine/Data-Science/master/Deep%20Learning%20SOTA/img/netconvgg.png)

## - Make the Model (VGG)

https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html

https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html

https://gaussian37.github.io/dl-concept-global_average_pooling/

In [17]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


# def test():
#     net = VGG('VGG11')
#     x = torch.randn(2,3,32,32)
#     y = net(x)
#     print(y.size())

# test()

In [18]:
model     = VGG('VGG11').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WD)
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 8, 8]         295,168
      BatchNorm2d-10            [-1, 256, 8, 8]             512
             ReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 256, 8, 8]         590,080
      BatchNorm2d-13            [-1, 256, 8, 8]             512
             ReLU-14            [-1, 25

In [10]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [11]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## - Train the Model

In [12]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.557156
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.415737
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.438557
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.322118
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.141386
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.099351
Train Epoch: 1 [30720/50000 (61%)]	Loss: 1.111422
Train Epoch: 1 [35840/50000 (71%)]	Loss: 0.972152
Train Epoch: 1 [40960/50000 (82%)]	Loss: 1.005072
Train Epoch: 1 [46080/50000 (92%)]	Loss: 0.936210
[1] Test Loss: 1.0175, Accuracy: 64.60%
Train Epoch: 2 [0/50000 (0%)]	Loss: 0.830754
Train Epoch: 2 [5120/50000 (10%)]	Loss: 0.754580
Train Epoch: 2 [10240/50000 (20%)]	Loss: 0.751464
Train Epoch: 2 [15360/50000 (31%)]	Loss: 0.770648
Train Epoch: 2 [20480/50000 (41%)]	Loss: 0.732383
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.707951
Train Epoch: 2 [30720/50000 (61%)]	Loss: 0.766757
Train Epoch: 2 [35840/50000 (71%)]	Loss: 0.861021
Train Epoch: 2 [40960/50000 (82%)]	Loss: 0.739399
Train Epoch: 2 [46080/

KeyboardInterrupt: 

### Hmm...